In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import pretrainedmodels as ptmodels
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection

#from ff import model_selection
from pipeline.metrics import accuracy_b, log_loss
from pipeline.model_methods import validate_img, train_img, validate_vid, train_vid
from pipeline.data_loaders import load_img_dataset, load_img_val_dataset, strong_aug
from pipeline.image_extracting import extract_faces, FastMTCNN

from pipeline.metrics import accuracy_sigmoid, accuracy_sigmoid_mean, log_loss_sigmoid, log_loss_b, accuracy_b_mean
from pipeline.model_methods import validate_vid_bf, train_vid_bf
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN
from pipeline.blazeface import BlazeFace
from efficientnet_pytorch import EfficientNet


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
fast_mtcnn = FastMTCNN(
    stride=4,
    resize=1,
    margin=14,
    factor=0.9,
    #min_face_size=60,
    #keep_all=True,
    select_largest=True,
    device=device
)
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
facedet = BlazeFace().to(device)
facedet.load_weights("pipeline/blazeface.pth")
facedet.load_anchors("pipeline/anchors.npy")
_ = facedet.train(False)

from pipeline.helpers.read_video_1 import VideoReader
from pipeline.helpers.face_extract_1 import FaceExtractor

frames_per_video = 17

video_reader = VideoReader()
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [4]:
y_train = pd.read_csv(r'D:\Machine Learning\deepfake-detection\data\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(100000, 19154)

In [5]:
#%%script false

batch_size = 16
normalize = torchvision.transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))

x_train = load_img_dataset(r'data\train', batch_size, resize=224, normalize=normalize)
x_val = load_img_val_dataset('data\img_val', 10, resize=224, normalize=normalize)

x_train_vid = r'D:\Machine Learning\deepfake-detection\data\train_set'
x_val_vid = r'data\val'
x_test_vid = r'D:\Machine Learning\deepfake-detection\data\cross_val'

In [6]:
class MyResNeXt(models.resnet.ResNet):
    def __init__(self, training=True):
        super(MyResNeXt, self).__init__(block=models.resnet.Bottleneck,
                                        layers=[3, 4, 6, 3], 
                                        groups=32, 
                                        width_per_group=4)
        #self.fc = nn.Linear(2048, 1)

In [7]:
model = EfficientNet.from_pretrained('efficientnet-b0')#models.resnext50_32x4d(pretrained=True) models.resnext101_32x8d(pretrained=True) MyResNeXt()
model._fc = nn.Linear(1280, 1)
'''
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(2048, 1)
)
'''

Loaded pretrained weights for efficientnet-b0


'\nmodel.fc = nn.Sequential(\n    nn.Dropout(0.5),\n    nn.Linear(2048, 1)\n)\n'

In [8]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [9]:
#%%script false

#model.load_state_dict(torch.load("pretrained/resnext50_32x4d-7cdf4587.pth", map_location=device))
model.load_state_dict(torch.load("0.8134302675729158 0.40084000225509364.pth", map_location=device))
_ = model.eval()

In [10]:
#model._fc = nn.Linear(1280, 1)

'''
model.fc = nn.Sequential(
                nn.Linear(2048, 256), 
                nn.ReLU(), 
                nn.Dropout(0.5),
                nn.Linear(256, 1))
'''

'\nmodel.fc = nn.Sequential(\n                nn.Linear(2048, 256), \n                nn.ReLU(), \n                nn.Dropout(0.5),\n                nn.Linear(256, 1))\n'

In [11]:
model = model.to(device)

In [12]:
loss =  torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: epoch // 30, lambda epoch: 0.95 ** epoch], last_epoch=-1)

In [13]:
i = 0
for child in model.children():
    print(i, '[')
    print(child)
    print(']')
    i += 1
print(i)

0 [
Conv2dStaticSamePadding(
  3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
  (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
)
]
1 [
BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
]
2 [
ModuleList(
  (0): MBConvBlock(
    (_depthwise_conv): Conv2dStaticSamePadding(
      32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_se_reduce): Conv2dStaticSamePadding(
      32, 8, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_se_expand): Conv2dStaticSamePadding(
      8, 32, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_project_conv): Conv2dStaticSamePadding(
      32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
      (static_padding): Identity()
    )


In [14]:
#%%script false
count = 0
for child in model.children():  
    count+=1
    if count < 0:    
        for param in child.parameters():
            param.requires_grad = False
    else:
        for param in child.parameters():
            param.requires_grad = True

In [15]:
[i[0] for i in model.named_parameters() if i[1].requires_grad]

['_conv_stem.weight',
 '_bn0.weight',
 '_bn0.bias',
 '_blocks.0._depthwise_conv.weight',
 '_blocks.0._bn1.weight',
 '_blocks.0._bn1.bias',
 '_blocks.0._se_reduce.weight',
 '_blocks.0._se_reduce.bias',
 '_blocks.0._se_expand.weight',
 '_blocks.0._se_expand.bias',
 '_blocks.0._project_conv.weight',
 '_blocks.0._bn2.weight',
 '_blocks.0._bn2.bias',
 '_blocks.1._expand_conv.weight',
 '_blocks.1._bn0.weight',
 '_blocks.1._bn0.bias',
 '_blocks.1._depthwise_conv.weight',
 '_blocks.1._bn1.weight',
 '_blocks.1._bn1.bias',
 '_blocks.1._se_reduce.weight',
 '_blocks.1._se_reduce.bias',
 '_blocks.1._se_expand.weight',
 '_blocks.1._se_expand.bias',
 '_blocks.1._project_conv.weight',
 '_blocks.1._bn2.weight',
 '_blocks.1._bn2.bias',
 '_blocks.2._expand_conv.weight',
 '_blocks.2._bn0.weight',
 '_blocks.2._bn0.bias',
 '_blocks.2._depthwise_conv.weight',
 '_blocks.2._bn1.weight',
 '_blocks.2._bn1.bias',
 '_blocks.2._se_reduce.weight',
 '_blocks.2._se_reduce.bias',
 '_blocks.2._se_expand.weight',
 '_bloc

In [16]:
%%script false

# default - Validation: metrics  0.6033333333333334 loss:  0.6769222617149353

validate_vid_bf(model, x_val_vid, y_train, log_loss_sigmoid, accuracy_sigmoid_mean, device, 2, face_extractor, 
                fast_mtcnn=fast_mtcnn,
                print_results=True, 
                reverse=False)

Couldn't find program: 'false'


In [17]:
#%%script false

#0.37783308817654526 center crop
#0.3752234482172356  256
#0.40455125311689993 224
#0.44303366757688983 299

validate_img(model, x_val, y_train, loss, accuracy_b, device, 10, 
             print_results=True, 
             inference=False,
             checkpoint=None,
             reverse=False
            )

tensor([-2.3456], device='cuda:0') tensor([0.], device='cuda:0')
tensor([5.0670], device='cuda:0') tensor([1.], device='cuda:0')
tensor([0.1866], device='cuda:0') tensor([1.], device='cuda:0')
tensor([7.5800], device='cuda:0') tensor([1.], device='cuda:0')
tensor([0.1055], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.1799], device='cuda:0') tensor([1.], device='cuda:0')
tensor([1.3547], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-3.2598], device='cuda:0') tensor([0.], device='cuda:0')
tensor([0.5392], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-3.7141], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-3.6457], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.6755], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.1343], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.6546], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-2.8698], device='cuda:0') tensor([0.], device='cuda:0')
tensor([1.9135], device='cuda:0

tensor([5.9754], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-2.1850], device='cuda:0') tensor([0.], device='cuda:0')
tensor([3.3348], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.3020], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.0327], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-2.4344], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-5.2222], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-3.2052], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-2.3220], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-2.6026], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-1.2628], device='cuda:0') tensor([0.], device='cuda:0')
tensor([1.1275], device='cuda:0') tensor([1.], device='cuda:0')
tensor([0.9039], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-2.6917], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-1.1901], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-2.5930], device='cud

tensor([1.1595], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-1.0283], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-1.7796], device='cuda:0') tensor([0.], device='cuda:0')
tensor([6.0676], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-2.3449], device='cuda:0') tensor([0.], device='cuda:0')
tensor([0.5309], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.8953], device='cuda:0') tensor([0.], device='cuda:0')
tensor([4.3842], device='cuda:0') tensor([1.], device='cuda:0')
tensor([1.0484], device='cuda:0') tensor([1.], device='cuda:0')
tensor([6.0134], device='cuda:0') tensor([1.], device='cuda:0')
tensor([7.2607], device='cuda:0') tensor([1.], device='cuda:0')
tensor([4.0627], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.8935], device='cuda:0') tensor([0.], device='cuda:0')
tensor([3.6383], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-3.4116], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-4.5267], device='cuda:0')

tensor([1.4926], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.2448], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.0213], device='cuda:0') tensor([0.], device='cuda:0')
tensor([4.7525], device='cuda:0') tensor([1.], device='cuda:0')
tensor([2.1555], device='cuda:0') tensor([1.], device='cuda:0')
tensor([3.5719], device='cuda:0') tensor([1.], device='cuda:0')
tensor([1.7473], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.2483], device='cuda:0') tensor([0.], device='cuda:0')
tensor([7.3630], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.8291], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-1.1344], device='cuda:0') tensor([0.], device='cuda:0')
tensor([2.7902], device='cuda:0') tensor([1.], device='cuda:0')
tensor([2.8900], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.5227], device='cuda:0') tensor([0.], device='cuda:0')
tensor([2.7320], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.7722], device='cuda:0')

tensor([4.7346], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.4457], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.2364], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.0951], device='cuda:0') tensor([1.], device='cuda:0')
tensor([0.4402], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.6527], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-4.5632], device='cuda:0') tensor([0.], device='cuda:0')
tensor([0.4570], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-4.7336], device='cuda:0') tensor([0.], device='cuda:0')
tensor([3.4809], device='cuda:0') tensor([1.], device='cuda:0')
tensor([0.7503], device='cuda:0') tensor([0.], device='cuda:0')
tensor([3.1906], device='cuda:0') tensor([1.], device='cuda:0')
tensor([2.6938], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.1204], device='cuda:0') tensor([0.], device='cuda:0')
tensor([0.6377], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-3.0174], device='cuda:0'

...

tensor([1.5696], device='cuda:0') tensor([1.], device='cuda:0')
tensor([7.0676], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.5411], device='cuda:0') tensor([0.], device='cuda:0')
tensor([5.2254], device='cuda:0') tensor([1.], device='cuda:0')
tensor([5.5001], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-3.0676], device='cuda:0') tensor([0.], device='cuda:0')
tensor([3.1596], device='cuda:0') tensor([1.], device='cuda:0')
tensor([0.3597], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.5214], device='cuda:0') tensor([1.], device='cuda:0')
tensor([4.7488], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.5549], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.0225], device='cuda:0') tensor([1.], device='cuda:0')
tensor([4.5778], device='cuda:0') tensor([1.], device='cuda:0')
tensor([0.1515], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-4.2737], device='cuda:0') tensor([0.], device='cuda:0')
tensor([4.4006], device='cuda:0') 

tensor([0.8688], device='cuda:0') tensor([1.], device='cuda:0')
tensor([3.6334], device='cuda:0') tensor([1.], device='cuda:0')
tensor([4.1441], device='cuda:0') tensor([1.], device='cuda:0')
tensor([6.2015], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.9911], device='cuda:0') tensor([0.], device='cuda:0')
tensor([0.0355], device='cuda:0') tensor([1.], device='cuda:0')
tensor([2.1173], device='cuda:0') tensor([1.], device='cuda:0')
tensor([0.3795], device='cuda:0') tensor([0.], device='cuda:0')
tensor([4.7096], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.9070], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.4771], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.1837], device='cuda:0') tensor([1.], device='cuda:0')
tensor([5.6039], device='cuda:0') tensor([1.], device='cuda:0')
tensor([3.5392], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-3.8246], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.2284], device='cuda:0') 

tensor([0.4609], device='cuda:0') tensor([1.], device='cuda:0')
tensor([5.5523], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-5.6464], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-1.3350], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-2.6917], device='cuda:0') tensor([0.], device='cuda:0')
tensor([3.3500], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.2432], device='cuda:0') tensor([0.], device='cuda:0')
tensor([7.2691], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.7208], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.2828], device='cuda:0') tensor([0.], device='cuda:0')
tensor([3.8725], device='cuda:0') tensor([1.], device='cuda:0')
tensor([2.9596], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-4.8052], device='cuda:0') tensor([0.], device='cuda:0')
tensor([1.9376], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-1.1861], device='cuda:0') tensor([0.], device='cuda:0')
tensor([0.2567], device='cuda:0'

tensor([3.0299], device='cuda:0') tensor([0.], device='cuda:0')
tensor([-0.0515], device='cuda:0') tensor([0.], device='cuda:0')
tensor([2.8425], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.3347], device='cuda:0') tensor([0.], device='cuda:0')
tensor([5.7006], device='cuda:0') tensor([1.], device='cuda:0')
tensor([1.1994], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-2.6281], device='cuda:0') tensor([0.], device='cuda:0')
tensor([0.9726], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-0.4387], device='cuda:0') tensor([0.], device='cuda:0')
tensor([1.4599], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-6.8846], device='cuda:0') tensor([0.], device='cuda:0')
tensor([0.1329], device='cuda:0') tensor([0.], device='cuda:0')
tensor([0.9271], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-1.7395], device='cuda:0') tensor([1.], device='cuda:0')
tensor([3.3018], device='cuda:0') tensor([1.], device='cuda:0')
tensor([-2.9040], device='cuda:0')

(0.8302933838079669, 0.37011126635215613)

In [20]:
0.37011126635215613 * 100 / 84, 0.37011126635215613 + 0.08228

(0.44060865041923347, 0.45239126635215615)

In [19]:
#%%script false
# 38% / 47% / 57% /58.5% / 61% - 0.656% (0.99344%) - video
# 84% undersampling / 50% oversampling / 89.8% original dataset / 0.08228

metric, lost = train_i1mg(model, loss, optimizer, scheduler, x_train, x_val, y_train, accuracy_b, device, [log_loss_sigmoid, accuracy_sigmoid_mean], face_extractor, 
                          epochs=20, 
                       batch_size=batch_size, 
                       del_net=False, 
                       useInference=False, 
                       inference=False, 
                       useScheduler=True,
                       checkpoint=[0.43, 0.99],
                         reverse=False
                      )

NameError: name 'train_i1mg' is not defined

In [ ]:
from operator import itemgetter
min_lost = max(enumerate(lost), key=itemgetter(1))[0] 
min_metrics = min(enumerate(metric), key=itemgetter(1))[0] 
print("Loss optim:", lost[min_lost], min_lost + 1)
print("Metrics optim:", metric[min_metrics], min_metrics + 1)

In [ ]:
plt.plot(metric, label='model')
plt.legend()
plt.title('Validation metrics')

In [ ]:
plt.plot(lost, label='model')
plt.legend()
plt.title('Validation Loss')

In [ ]:
'''model = torch.load(PATH)
model = model.to(device)
model.eval()'''

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, print_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_results=True, inference=nn.Softmax(dim=1))

In [ ]:
validate_img(model, x_val, y_train, loss, log_loss, device, batch_size, show_graphic=True, inference=nn.Softmax(dim=1))

torch.save(model.state_dict(), 'model.pth')